# Home Credit Default Risk
Home Credit is an international non-bank, consumer finance group. The company operates in 14 countries and focuses on lending primarily to people with little or no credit history.

*Credit* is the trust which allows one party to provide money or resources to another party where that second party does not reimburse the first party immediately (*thereby generating a debt*), but instead promises either to repay or return those resources at a later date. A *credit bureau* is a collection agency that gathers account information from various creditors.

This notebook aims to be able to **predict how capable an applicant is of repaying a loan**, using datasets, provided by Home Credit.

[Link](https://www.kaggle.com/c/home-credit-default-risk/data) to the datasets.

## Attributes

A **cash loan** is the lending of money by one or more individuals, organizations, or other entities to other individuals, organizations etc. The recipient (i.e. the borrower) incurs a debt, and is usually liable to pay interest on that debt until it is repaid, and also to repay the principal amount borrowed.  
**Revolving credit/loan** is a type of credit that does not have a fixed number of payments, in contrast to installment credit. Credit cards are an example of revolving credit used by consumers.


In [16]:
# Manipulating data
import numpy as np
import pandas as pd

# Managing files
import os

## The different datasets

* **application_train.csv** and **application_test.csv**
  * The main table in two files. The train set contains __labels/targets__ (0: the loan was *repaid* or 1: the loan was *not* repaid), and the test set does not.
  * Static data for all applications. One row represents one loan.  
  
  
* **bureau.csv**
  * All client's previous credits provided by other financial institutions reported by the Credit Bureau (loans).
  * For every loan, there are as many rows as number of credits the client had in Credit Bureau before the application date. In other words, each previous credit has its own row in bureau, but one loan in the application data can have multiple previous credits.
  
  
* **bureau_balance.csv**
  * Monthly balances of previous credits in Credit Bureau (**bureau.csv**).
  * This table has one row for each month of history of every previous credit reported to Credit Bureau - the table has (*# loans in sample * # of relative previous credits * # of months where we have some history observable for the previous credits*) rows. In other words, each row is one month of a previous credit, and a single previous credit can have multiple rows, *one for each month* of the credit length.
  
  
* **POS_CASH_balance.csv**
  * Monthly balance snapshots/data of previous **POS (point of sales) and cash loans** that the applicant had with Home Credit.
  * This table has one row for each month of history of every previous credit in Home Credit (consumer credit and cash loans) related to loans in our sample – i.e. the table has (*# loans in sample * # of relative previous credits * # of months in which we have some history observable for the previous credits*) rows.
  
  
* **credit_card_balance.csv**
  * Monthly balance snapshots of previous **credit cards** that the applicant has with Home Credit.
  * This table has one row for each month of history of every previous credit in Home Credit (consumer credit and cash loans) related to loans in our sample - the table has (*# loans in sample * # of relative previous credit cards * # of months where we have some history observable for the previous credit card*) rows. In other words, each row is one month of a credit card balance, and a single credit card can have many rows.
  
  
* **previous_application.csv**
  * All previous applications for Home Credit loans of *clients who have loans* in our sample.
  * There is one row for each previous application related to loans in our data sample. Each current loan in the application data can have multiple previous loans. Each previous application has one row and is identified by the feature SK_ID_PREV. 
  
  
* **installments_payments.csv** (*A sum of money paid in small parts in a fixed period of time or a single payment within a staged payment plan of a loan*)
  * Repayment history for the previously disbursed credits in Home Credit related to the loans in our sample.
  * There is a) one row for **every payment that was made** plus b) one row each for **missed payment**.
  * One row is equivalent to one payment of one installment OR one installment corresponding to one payment of one previous Home Credit credit related to loans in our sample.
  
  
* **HomeCredit_columns_description.csv**
  * This file contains descriptions for the columns in the various data files.

In [17]:
![the datasets](misc/datasets.png)

/bin/sh: 1: Syntax error: "(" unexpected


In [18]:
print(os.listdir("datasets/"))

['credit_card_balance.csv', 'HomeCredit_columns_description.csv', 'installments_payments.csv', 'sample_submission.csv', 'bureau.csv', 'previous_application.csv', 'POS_CASH_balance.csv', 'application_train.csv', 'application_test.csv', 'bureau_balance.csv']


In [19]:
train_set = pd.read_csv('datasets/application_train.csv')
test_set = pd.read_csv('datasets/application_test.csv')

## Exploring the data

Exploring the data to learn more about the data we have available.

In [20]:
print("The training set contains the following number of rows and columns", train_set.shape)

The training set contains the following number of rows and columns (307511, 122)


In [21]:
train_set.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
list(train_set)

['SK_ID_CURR',
 'TARGET',
 'NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'CNT_CHILDREN',
 'AMT_INCOME_TOTAL',
 'AMT_CREDIT',
 'AMT_ANNUITY',
 'AMT_GOODS_PRICE',
 'NAME_TYPE_SUITE',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'REGION_POPULATION_RELATIVE',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'DAYS_REGISTRATION',
 'DAYS_ID_PUBLISH',
 'OWN_CAR_AGE',
 'FLAG_MOBIL',
 'FLAG_EMP_PHONE',
 'FLAG_WORK_PHONE',
 'FLAG_CONT_MOBILE',
 'FLAG_PHONE',
 'FLAG_EMAIL',
 'OCCUPATION_TYPE',
 'CNT_FAM_MEMBERS',
 'REGION_RATING_CLIENT',
 'REGION_RATING_CLIENT_W_CITY',
 'WEEKDAY_APPR_PROCESS_START',
 'HOUR_APPR_PROCESS_START',
 'REG_REGION_NOT_LIVE_REGION',
 'REG_REGION_NOT_WORK_REGION',
 'LIVE_REGION_NOT_WORK_REGION',
 'REG_CITY_NOT_LIVE_CITY',
 'REG_CITY_NOT_WORK_CITY',
 'LIVE_CITY_NOT_WORK_CITY',
 'ORGANIZATION_TYPE',
 'EXT_SOURCE_1',
 'EXT_SOURCE_2',
 'EXT_SOURCE_3',
 'APARTMENTS_AVG',
 'BASEMENTAREA_AVG',
 'YEARS_BEGINEXPLUATATION_A

In [23]:
train_set.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
print("The training set contains the following number of rows and columns", test_set.shape)

The training set contains the following number of rows and columns (48744, 121)


In [25]:
test_set.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


### Repaid vs not repaid

Repaid loans (0) compared to not repaid loans (1) shows us that, *there are far more repaid loans than not repaid loans*. Most machine learning algorithms work best when the number of classes are roughly equal. This is called the *class imbalance problem*. We want to emphasize to the machine learning algorithm that it's important to detect the few cases where a loan is likely not to be repaid, rather than focusing only on high accuracy.

In [27]:
train_set["TARGET"].value_counts()

0    282686
1     24825
Name: TARGET, dtype: int64